# Address missing data

In [1]:
import urllib
import urllib.request
import urllib.parse
import urllib.error
import requests
from bs4 import BeautifulSoup
import csv
import os
import pandas as pd
import clean_documents
from pathlib import Path
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from functools import reduce
import extract_dates
from start import data_path

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
exemptions_df = pd.read_csv(os.path.join(data_path, 'doi_exemptions_scraped.csv'))
dates_df = pd.read_csv(os.path.join(data_path, 'doi_dates_scraped.csv'))

In [3]:
docs_df = exemptions_df.merge(dates_df)
docs_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,title,level,type,link,text,p_innovation,...,term_phrase,term_p,finalize_year,finalize_month,finalize_phrase,finalize_p,date_year,date_month,date_phrase,date_ay
0,0,672,672,672,Lake Travis ISD,Second,pdf,https://www.ltisdschools.org//cms/lib/Tx018000...,"Effective Dec. 13, 2016 – Dec. 13, 2021 2 What...",0.999955,...,: 1. Beginning with the 2017-2018 academic yea...,0.999955,2016,December,"hold a public meeting on December 13, 2016 to ...",0.996723,2017.0,"(999, '')",: 1. Beginning with the 2017-2018 academic yea...,2017.0
1,1,1243,1243,1243,Zephyr ISD,Second,docx,http://zephyrisd.net/wp-content/uploads/2014/0...,Zephyr ISD District of Innovation Plan Introdu...,0.999922,...,Term The District of Innovation Plan will beco...,0.999955,2018,April,"go to the Board on April 16th, 2018. Term The ...",0.889274,2018.0,"(2018, 'September')",Term The District of Innovation Plan will beco...,2018.0
2,2,315,315,315,Zavalla ISD,First,pdf,http://www.zavallaisd.org/Uploads/269/misc/f28...,Zavalla ISD District of Innovation Plan (HB 18...,0.999619,...,This plan will be in effect for the 2017-2018 ...,0.999955,2017,February,"27,2017 Final version plan posted January 27, ...",0.999955,2017.0,"(999, '')",This plan will be in effect for the 2017-2018 ...,2017.0
3,3,3309,3309,3309,Zapata County ISD,Second,pdf,https://1.cdn.edl.io/VZ6fi1M6pn8Qk8tS3GetGewCD...,Microsoft Word - Document1 ZAPATA COUNTY INDEP...,0.452118,...,"a District of Innovation on Tuesday, December ...",0.000187,2017,December,"a District of Innovation on Tuesday, December ...",0.999884,2017.0,December,"a District of Innovation on Tuesday, December ...",2017.0
4,4,1196,1196,1196,Yorktown ISD,Second,pdf,http://www.yisd.org/userfiles/57/my%20files/fi...,Yorktown Independent School District Final Dis...,0.999877,...,Yorktown Independent School District Final Dis...,0.999955,2017,May,"of the letter. On May 15, 2017 the District Co...",0.972258,2017.0,"(999, '')",Yorktown Independent School District Final Dis...,2017.0
5,5,334,334,334,Yoakum ISD,First,pdf,http://www.yoakumisd.net/cms/lib3/TX01001553/C...,DISTRICT OF INNOVATION DISTRICT OF INNOVATION ...,0.998961,...,the plan will begin in the fall of 2018 and co...,0.999955,-999,NaN,NaN,0.000000,2018.0,"(999, '')",the plan will begin in the fall of 2018 and co...,2018.0
6,6,3667,3667,324,Wylie ISD (221912),html,html,http://www.wyliebulldogs.org/cms/One.aspx?port...,District of Innovation - Wylie Independent Sch...,0.012857,...,Procedures In-Service Emergency Procedures Sig...,0.999151,-999,NaN,Hall of Honor Nomination Required Postings Aca...,0.999955,2018.0,"(999, '')",Procedures In-Service Emergency Procedures Sig...,2018.0
7,7,276,276,276,Wylie ISD (043914),First,pdf,http://www.wylieisd.net/cms/lib09/TX01918453/C...,Wylie ISD INNOVATION PLAN INTRODUCTION House B...,0.999891,...,INNOVATION PLAN INTRODUCTION House Bill (HB) 1...,0.134219,2017,March,"Plan at its meeting on March 20, 2017. DISTRIC...",0.999955,2017.0,March,"Plan at its meeting on March 20, 2017. DISTRIC...",2016.0
8,8,1542,1542,1542,Wortham ISD,Second,docx,https://s3.amazonaws.com/scschoolfiles/888/wis...,W Wortham ISD District of Innovation Plan 2017...,0.997949,...,W Wortham ISD District of Innovation Plan 2017...,0.999955,-999,NaN,NaN,0.000000,2017.0,"(999, '')",W Wortham ISD District of Innovation Plan 2017...,2017.0
9,9,350,350,350,Woodville ISD,First,pdf,https://core-docs.s3.amazonaws.com/documents/a...,No text,0.001259,...,NaN,0.000000,-999,NaN,NaN,0.000000,-999.0,NaN,NaN,-999.0


## Check TEA website for new districts of innovation

In [4]:
add_new_dois = True

In [5]:
if add_new_dois:
    url = "https://tea.texas.gov/Texas_Schools/District_Initiatives/Districts_of_Innovation/"
    webcontent = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(webcontent, 'html.parser')
    links = soup.find_all('a')
    districts_list = [i for i in map(lambda x: x.get('title'), links) 
                      if i is not None and 'ISD' in i]
    districts_df = pd.DataFrame(districts_list, columns=['title'])
    len(districts_df)
    # Make a new DataFrame from a right join on dois with list of districts
    combined_df = docs_df.merge(districts_df, how='right', left_on='title', right_on='title')

    # Save the district name and link for those districts that we don't have laws for
    missing_districts = combined_df[(combined_df['p_innovation'].isna()) |
                       (combined_df['possible_laws'] == '[]')][['title', 'link']]

    print("Number of missing districts: ", len(missing_districts))
    missing_districts.head()

Number of missing districts:  218


In [6]:
if add_new_dois == False: 
    # Save the district name and link for those districts that we don't have laws for
    missing_districts = docs_df[(docs_df['p_innovation'].isna()) |
                       (docs_df['possible_laws'] == '[]')][['title', 'link']]
    print("Number of missing districts: ", len(missing_districts))
    missing_districts.head()

## Address districts that are missing due to incorrect link.

### Where possible, incorporate previously (manually) collected links before editing

In [7]:
missing_districts = missing_districts.rename({'link': 'link_scraped'}, axis = 'columns')
missing_links = pd.read_csv(os.path.join(data_path, 'missing_links_corrected.csv'))
missing_links = missing_links.rename({'link': 'link_manual'}, axis = 'columns')
missing_links = missing_districts.merge(missing_links, how = 'left', left_on = 'title', right_on = 'title')
print("Number of missing links", len(missing_links))
missing_links['link'] = missing_links.link_manual
missing_links['link'] = missing_links['link'].fillna(missing_links['link_scraped'])
missing_links.head()

Number of missing links 218


,title,link_scraped,Unnamed: 0,link_manual,link
0,Wylie ISD (221912),http://www.wyliebulldogs.org/cms/One.aspx?port...,0.0,http://www.wyliebulldogs.org/cms/One.aspx?port...,http://www.wyliebulldogs.org/cms/One.aspx?port...
1,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,1.0,https://core-docs.s3.amazonaws.com/documents/a...,https://core-docs.s3.amazonaws.com/documents/a...
2,Woodson ISD,https://sites.google.com/a/woodsonisd.net/wood...,2.0,https://drive.google.com/file/d/0B5Y_iCwf_UMuT...,https://drive.google.com/file/d/0B5Y_iCwf_UMuT...
3,Woden ISD,http://www.wodenisd.org/home/district-announce...,3.0,https://drive.google.com/file/d/1j-lifioD9y4Z7...,https://drive.google.com/file/d/1j-lifioD9y4Z7...
4,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,4.0,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...


In [8]:
missing_links = missing_links[['title', 'link']]
missing_links.to_csv(os.path.join(data_path, 'missing_links.csv'))

### Manually edit links in links_nolaws where wrong. Save as links_nolaws_corrected.csv. Then upload.

In [12]:
missing_links = pd.read_csv(os.path.join(data_path, 'missing_links_corrected.csv'))
missing_links.tail()

,Unnamed: 0,title,link
142,142,Anahuac ISD,https://drive.google.com/file/d/11uTSTuAcBFTdq...
143,143,Amarillo ISD,http://www.amaisd.org/UserFiles/Servers/Server...
144,144,Garner ISD,https://docs.google.com/a/garnerisd.net/viewer...
145,145,Temple ISD,https://4.files.edl.io/6f40/08/17/18/230315-72...
146,146,Yantis ISD,http://www.yantisisd.net/users/2017-2018/Distr...


### Extract text from missing_links

In [13]:
texts = []
for link in missing_links.link:
    text = clean_documents.get_plain_text(link)
    texts.append(text)

2018-09-08 10:07:37,705 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.18/tika-server-1.18.jar to /var/folders/qd/n3w8bh_14j9c4rs4qpn5vjq00000gn/T/tika-server.jar.
2018-09-08 10:07:40,765 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.18/tika-server-1.18.jar.md5 to /var/folders/qd/n3w8bh_14j9c4rs4qpn5vjq00000gn/T/tika-server.jar.md5.
2018-09-08 10:07:41,117 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Google Error: 'd' is not in listle.com/drive/folders/0B-ALJgmFKVDFZmxBb09rRE9aQ1UD%20Final%20DOI%20Plan.pdficity/Domain/6734/2017.03.16-WacoISD-DistrictofInnovation-Exemptions-BoardApproved.pdf
Google Error: 'd' is not in listle.com/open?id=1kC4Rn_OgoK8K-s3kvUh4xBPTluPJdtpKtes/News/Documents/FINAL_DOI_PDF_FIle.pdf20Final.pdf20Innovation%20Plan%20June%2012%202017.pdf
Google Error: 'd' is not in listle.com/drive/folders/169mMXgf5g3TeRrsjYPg7fpq25FI57Ce_?usp=sharing_2017.pdfe=News018-2023.pdfistrict+of+Innovation+Plan.pdf+2018.docx5Forg%2FDocuments%2F2018%20Website%20%2D%20Required%2FDistrict%2Dof%2DInnovation%2DPlan%2D2016%2D2017%2DBoard%2DApproved%2Epdf&parent=%2Fpersonal%2Fmarcus%5Fdavis%5Fpottsboroisd%5Forg%2FDocuments%2F2018%20Website%20%2D%20Required&slrid=58be829e%2D30c5%2D6000%2D8997%2De55bfde53779


2018-09-08 10:10:33,481 [MainThread  ] [WARNI]  Tika server returned status: 422


error: HTTP Error 404: File Not Found link: http://www.ezzellisd.org/apps/pages/index.jsp?uREC_ID=199567&type=d&pREC_ID=431083
Google Error: 'd' is not in listle.com/open?id=0B91INW8X-t0FQ1JVT0dmYnIxcGM8VZTmK2E.pdfr%20Colmesneil%20ISD%20%281%29.pdf20District%20of%20Innovation%203-21-17.pdf
Google Error: 'd' is not in listle.com/drive/folders/0B3zKmpprcZY1OHRBcEw1d2VFcFE


In [14]:
missing_links['text'] = texts
missing_links['text'] = missing_links['text'].fillna('No text')
len(missing_links)

147

In [15]:
missing_links = clean_documents.remove_whitespace(missing_links, text_col = 'text')
missing_links.head()

,Unnamed: 0,title,link,text
0,0,Wylie ISD (221912),http://www.wyliebulldogs.org/cms/One.aspx?port...,District of Innovation - Wylie Independent Sch...
1,1,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text
2,2,Woodson ISD,https://drive.google.com/file/d/0B5Y_iCwf_UMuT...,Woodson Independent School District District o...
3,3,Woden ISD,https://drive.google.com/file/d/1j-lifioD9y4Z7...,DISTRICT OF INNOVATION PLAN Introduction The 8...
4,4,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...


### Extract laws from missing_links

In [16]:
nlp = spacy.load('en_core_web_sm')
law_shapes = [i*'d' + '.' + j*'d' for i in range(1, 4) for j in range(3,5)]
law_shape_patterns = [[{'LIKE_NUM':True, 'SHAPE':shape}, {'ORTH':'%', 'OP':'!'}] for shape in law_shapes] # could add {'SHAPE':'§', 'OP':'*'},  and exceptions for laws in parentheses or followed by a hyphen

matcher = Matcher(nlp.vocab)
matcher.add("ExplicitLaw", None, *law_shape_patterns)

def get_matches(string):
    doc = nlp(string)
    matches = matcher(doc)
    return list(set([doc[i[1]:i[2]][0] for i in matches]))

missing_links['possible_laws'] = missing_links.text.apply(get_matches)

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


### Extract dates from manually collected links

In [17]:
output_dir = os.path.join(data_path, 'date_term_classifier')
start_dates = []
date_phrases = []
p_terms = []
for text in missing_links.text:
    date, phrase, p = extract_dates.get_term_date_and_phrase(text, output_dir)
    start_dates.append(date)
    date_phrases.append(phrase)        
    p_terms.append(p)
missing_links['date'] = start_dates
missing_links['date_phrase'] = date_phrases
missing_links['date_p'] = p_terms

In [18]:
len(missing_links)

147

In [19]:
print('There are', len(missing_links[missing_links.text == 'No text']), 'documents without text (photocopies, most likely)')
print('There are', len(missing_links[missing_links.text == 'UNAVAILABLE']), 'documents which we cannot access')
print('An additional', len(missing_links[missing_links['possible_laws'].astype(str) == '[]'][missing_links.text != 'No text']), 'documents have text but do not contain laws.')
print('and', len(missing_links[missing_links['date'] == -999][missing_links.text != 'No text']), 'documents have text but do not contain dates.')

There are 52 documents without text (photocopies, most likely)
There are 1 documents which we cannot access
An additional 65 documents have text but do not contain laws.
and 45 documents have text but do not contain dates.


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


### Save now correct observations to 'missing_links_corrected'. Save districts that are still incorrect to missing_text.

In [20]:
missing_links_corrected = missing_links[missing_links['possible_laws'].astype(str) != '[]']
missing_text = missing_links[missing_links['possible_laws'].astype(str) == '[]']

In [21]:
len(missing_links)

147

In [22]:
print('We have added', len(missing_links_corrected), 'districts.')
print(len(missing_text), 'are still missing.')

We have added 30 districts.
117 are still missing.


### Export districts with missing or incorrect text. 

In [23]:
missing_text.to_csv(os.path.join(data_path, 'missing_text.csv'))

## Address districts that are missing due to missing or incorrect text.

##### Manually add correct text. If no text (photocopy), put 'No text' in cell. If you cannot find the document, put 'No document found'. Save as missing_text_edited.csv.

In [24]:
missing_texts = pd.read_csv(os.path.join(data_path, 'missing_text_corrected.csv'))

### Extract laws

In [25]:
missing_texts['possible_laws'] = missing_texts.text.apply(get_matches)

### Extract dates

In [26]:
output_dir = os.path.join(data_path, 'date_term_classifier')
start_dates = []
date_phrases = []
p_terms = []
for text in missing_texts.text:
    date, phrase, p = extract_dates.get_term_date_and_phrase(text, output_dir)
    start_dates.append(date)
    date_phrases.append(phrase)        
    p_terms.append(p)
missing_texts['date'] = start_dates
missing_texts['date_phrase'] = date_phrases
missing_texts['date_p'] = p_terms

In [27]:
missing_texts.head()

,Unnamed: 0,Unnamed: 0.1,title,link,text,possible_laws,date,date_phrase,date_p
0,0,0,Wylie ISD (221912),http://www.wyliebulldogs.org/cms/One.aspx?port...,District of Innovation Introduction HB 1842 wa...,"[25.112, 25.0811, 21.003, 25.113, 21.057, 21.0...",2015,during the 84th Texas Legislative Session in S...,0.000243
1,1,1,Woodville ISD,https://core-docs.s3.amazonaws.com/documents/a...,No text,[],-999,,0.000000
2,4,4,Wink-Loving ISD,https://1.cdn.edl.io/vQwTZvpBxb8PX0K2KFrGbN9Y0...,Wink-Loving ISD WLISD - The Best in Texas: Bui...,[],-999,the 84 th Legislative Session in House Bill 18...,0.002479
3,5,5,Windthorst ISD,http://www.windthorstisd.net/Uploads/50/misc/f...,No text,[],-999,,0.000000
4,6,6,Willis ISD,https://tx50000123.schoolwires.net//cms/lib/TX...,No text,[],-999,,0.000000


In [28]:
missing_text_corrected = missing_texts[missing_texts['possible_laws'].astype(str) != '[]']
missing_laws = missing_texts[missing_texts['possible_laws'].astype(str) == '[]']

In [29]:
print('We have added', len(missing_text_corrected), 'districts.')
print(len(missing_laws), 'are still missing.')

We have added 25 districts.
92 are still missing.


# Address districts that are photocopies

In [30]:
missing_laws[['title', 'possible_laws', 'date', 'link']].to_csv(os.path.join(data_path, 'missing_laws_and_dates.csv'))

In [31]:
missing_laws_corrected = pd.read_csv(os.path.join(data_path, 'missing_laws_and_dates_corrected.csv'))

# Update full dataset

In [32]:
missing_links_corrected = missing_links_corrected.set_index('title')
missing_text_corrected = missing_text_corrected.set_index('title')
missing_laws_corrected = missing_laws_corrected.set_index('title')
docs_df = docs_df.set_index('title')
docs_df = docs_df.sort_index()

In [33]:
docs_df.update(missing_links_corrected)
docs_df.update(missing_text_corrected)
docs_df.update(missing_laws_corrected)

In [34]:
print("Only missing", len(docs_df[docs_df['possible_laws'].astype(str) == '[]']), " of ", len(docs_df),  "- unable to find plans.")

Only missing 12  of  781 - unable to find plans.


# Save

In [35]:
doi_dates_and_exemptions = docs_df.to_csv(os.path.join(data_path, 'doi_dates_and_exemptions_temp.csv'))